In [1]:
# !git clone https://github.com/salesforce/awd-lstm-lm.git

In [2]:
import argparse
import time
import math
import numpy as np
np.random.seed(331)
import torch
import torch.nn as nn
import os, sys

sys.path.append('/home/amank/Documents/projects/ulmfit/awd-lstm-lm/')

from torch.autograd import Variable
import data
import model
from embed_regularize import embedded_dropout
from locked_dropout import LockedDropout
from weight_drop import WeightDrop
from utils import batchify, get_batch, repackage_hidden
import hashlib
import pickle

In [3]:
!pwd

/home/amank/Documents/projects/ulmfit


In [6]:
# !python awd-lstm-lm/main.py --batch_size 20 --data /home/amank/Documents/projects/ulmfit/awd-lstm-lm/data/ptb --dropouti 0.4 --dropouth 0.25 --seed 141 --epoch 500 --save PTB.pt

In [4]:
randomhash = ''.join(str(time.time()).split('.'))

# ntokens - what is this? - I believe this denotes the number of strings in wikiText-103
args_data='/home/amank/Documents/projects/ulmfit/awd-lstm-lm/data/ptb/'
args_model='LSTM'
args_embedSize=400 
args_nhid=1150
args_nlayers=3
args_lr=30
args_clip=0.25
args_epochs=8000
# args_batchSize=80 # for WT-2
args_batchSize=40 # for PTB dataset
args_bptt=70
args_dropout=0.4 
args_dropouth=0.3
args_dropouti=0.65
args_dropoute=0.1 
args_wdrop=0.5 
args_tied=True
args_seed=1111
args_nonMono=5
args_cuda=True # 'store_false'
args_logInterval=200
args_save=randomhash+'.pt'
args_alpha=2
args_beta=1
args_wDecay=1.2e-6

In [5]:
obj = pickle.load(open(args_data+'itos_wt103.pkl', "rb"))
args_numTokens=len(obj)

In [6]:
model2 = model.RNNModel(args_model, args_numTokens, args_embedSize, args_nhid, args_nlayers, args_dropout, args_dropouth, args_dropouti, args_dropoute, args_wdrop, args_tied)

Applying weight drop of 0.5 to weight_hh_l0
Applying weight drop of 0.5 to weight_hh_l0
Applying weight drop of 0.5 to weight_hh_l0
[WeightDrop(
  (module): LSTM(400, 1150)
), WeightDrop(
  (module): LSTM(1150, 1150)
), WeightDrop(
  (module): LSTM(1150, 400)
)]


In [7]:
total_params = sum(x.size()[0] * x.size()[1] if len(x.size()) > 1 else x.size()[0] for x in model2.parameters())
print('Model total parameters:', total_params)

Model total parameters: 115834862


In [8]:
# pretrained from ptb word level dataset
pretrained_model_weights=torch.load('/home/amank/Documents/projects/ulmfit/'+'converted_model.pth')
model2_dict = model2.state_dict()

# # 1. filter out unnecessary keys
pretrained_dict = {k: v for k, v in pretrained_model_weights.items() if k in model2_dict}
# # 2. overwrite entries in the existing state dict
model2_dict.update(pretrained_dict) 
# # 3. load the new state dict
model2.load_state_dict(model2_dict)

In [16]:
print(model2.named_modules)

<bound method Module.named_modules of RNNModel(
  (lockdrop): LockedDropout()
  (idrop): Dropout(p=0.65)
  (hdrop): Dropout(p=0.3)
  (drop): Dropout(p=0.4)
  (encoder): Embedding(238462, 400)
  (rnns): ModuleList(
    (0): WeightDrop(
      (module): LSTM(400, 1150)
    )
    (1): WeightDrop(
      (module): LSTM(1150, 1150)
    )
    (2): WeightDrop(
      (module): LSTM(1150, 400)
    )
  )
  (decoder): Linear(in_features=1150, out_features=238462, bias=True)
)>


In [39]:
# print(model2.decoder.parameters())
model2.lockdrop.parameters()
model2.idrop.parameters()
model2.hdrop.parameters()
model2.drop.parameters()
model2.encoder.parameters()
model2.decoder.parameters()

<generator object Module.parameters at 0x7fb3586f3518>


In [23]:
print(model2.rnns.parameters)

<bound method Module.parameters of ModuleList(
  (0): WeightDrop(
    (module): LSTM(400, 1150)
  )
  (1): WeightDrop(
    (module): LSTM(1150, 1150)
  )
  (2): WeightDrop(
    (module): LSTM(1150, 400)
  )
)>


In [ ]:
optimizer = torch.optim.SGD([
            {'params': model2.rnns[0].parameters(), 'lr': slant_lr/(2.6**2)},
            {'params': model2.rnns[1].parameters(), 'lr': slant_lr/2.6},
            {'params': model2.rnns[2].parameters(), 'lr': slant_lr}]) 
# , lr=opt.lr*0.1, momentum=0.9)
# {'params': model.fc.parameters(), 'lr': slant_lr}


In [ ]:
# ignored_params = list(map(id, model.fc.parameters()))
# base_params = filter(lambda p: id(p) not in ignored_params,
#                      model.parameters())

# optimizer = torch.optim.SGD([
#             {'params': base_params},
#             {'params': model.fc.parameters(), 'lr': opt.lr}
#         ], lr=opt.lr*0.1, momentum=0.9)

In [40]:
# for name, param in model2.named_parameters():
    # if param.requires_grad:
    # print(name, param.data)

In [10]:
# model2.eval()
# corpus = data.Corpus(args_data)
# ntokens = len(corpus.dictionary)
# hidden1 = model2.init_hidden(1)
# with torch.no_grad(): 
    # input1 = Variable(torch.rand(1, 1).mul(ntokens).long())
    # input1=torch.tensor(torch.rand(1,1).mul(ntokens).long())
# print("printing input and hidden below")
# print(input1)
# print(hidden1)

In [11]:
# output1, hidden1 = model2(input1, hidden1)

In [12]:
# print(type(output1))
# print(type(hidden1))

In [38]:
###############################################################################
# Language Modeling Inference using pretrained wiki-103 weights
#
# This file generates new sentences sampled from the language model
#
###############################################################################

# import argparse

# import torch
from torch.autograd import Variable

# import data

# parser = argparse.ArgumentParser(description='PyTorch PTB Language Model')

# Model parameters.
# Model parameters.
sample_data=args_data
sample_model='LSTM'
sample_seed=1111
sample_outf='generated.txt'
sample_words=1000
sample_temperature=1.0
sample_log_interval=100
sample_checkpoint='./sample_model.pt'
torch.manual_seed(sample_seed)
# parser.add_argument('--cuda', action='store_true',
                    # help='use CUDA')
# parser.add_argument('--temperature', type=float, default=1.0,
                    # help='temperature - higher will increase diversity')
# parser.add_argument('--log-interval', type=int, default=100,
                    # help='reporting interval')

# Set the random seed manually for reproducibility.
# torch.manual_seed(args_seed)

# if torch.cuda.is_available():
#     if not args.cuda:
#         print("WARNING: You have a CUDA device, so you should probably run with --cuda")
#     else:
#         torch.cuda.manual_seed(args.seed)

# if args.temperature < 1e-3:
#     parser.error("--temperature has to be greater or equal 1e-3")

# with open(args.checkpoint, 'rb') as f:
    # model = torch.load(f)

model2.eval()
# if args.model == 'QRNN':
#     model.reset()

# if args.cuda:
#     model.cuda()
# else:
#     model.cpu()

corpus = data.Corpus(sample_data)
ntokens = len(corpus.dictionary)
hidden = model2.init_hidden(1)
input = Variable(torch.rand(1, 1).mul(ntokens).long(), volatile=True)

with open(sample_outf, 'w') as outf:
    for i in range(sample_words):
        output, hidden = model2(input, hidden)
        word_weights = output.squeeze().data.div(sample_temperature).exp().cpu()
        word_idx = torch.multinomial(word_weights, 1)[0]
        input.data.fill_(word_idx)
        word = corpus.dictionary.idx2word[word_idx]

        outf.write(word + ('\n' if i % 20 == 19 else ' '))

        if i % sample_log_interval == 0:
            print('| Generated {}/{} words'.format(i, sample_words))

/home/amank/Documents/projects/ulmfit/venv/lib/python3.6/site-packages/ipykernel_launcher.py:62: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.


| Generated 0/1000 words
| Generated 100/1000 words
| Generated 200/1000 words
| Generated 300/1000 words
| Generated 400/1000 words
| Generated 500/1000 words
| Generated 600/1000 words
| Generated 700/1000 words
| Generated 800/1000 words
| Generated 900/1000 words


In [ ]:
# contd with task_3.2
# Finetuning LM --> discriminative fine-tuning & slanted triangular learning rates

In [1]:
# 11/21/2K18
# awd_lstm_lm finetune.py below

In [4]:
# torch.cuda.is_available()

True

In [5]:
def batchify2(data, bsz, args_cuda):
    # Work out how cleanly we can divide the dataset into bsz parts.
    nbatch = data.size(0) // bsz
    # Trim off any extra elements that wouldn't cleanly fit (remainders).
    data = data.narrow(0, 0, nbatch * bsz)
    # Evenly divide the data across the bsz batches.
    data = data.view(bsz, -1).t().contiguous()
    if args_cuda:
        data = data.cuda()
    return data


def get_batch2(source, i, args_bptt, seq_len=None, evaluation=False):
    seq_len = min(seq_len if seq_len else args_bptt, len(source) - 1 - i)
    data = source[i:i+seq_len]
    target = source[i+1:i+1+seq_len].view(-1)
    return data, target

In [27]:
# slant_lr=1e-2
# optimizerTest = torch.optim.SGD([
#             {'params': model2.rnns[0].parameters(), 'lr': slant_lr/(2.6**2)},
#             {'params': model2.rnns[1].parameters(), 'lr': slant_lr/2.6},
#             {'params': model2.rnns[2].parameters(), 'lr': slant_lr}]) 

In [29]:
for i in optimizerTest.param_groups:
    print(i['lr'])

0.0014792899408284023
0.003846153846153846
0.01


In [ ]:
np.random.seed(331)

# parser = argparse.ArgumentParser(description='PyTorch PennTreeBank RNN/LSTM Language Model')

# data(train, valid, test files)_path, model_type, embeddingsize, nhid1150, nlayers, lr, clip, epochs=8000, batchsize, bptt, 
# dropout, dropouth, dropouti, dropoute, wdrop, tied, seed1111, nonmono5, log_interval, randomhash_save, alpha, beta, 
# wdecay1.2e-6

# Set the random seed manually for reproducibility.
# torch.manual_seed(args.seed)
if torch.cuda.is_available():
    if not args_cuda:
        print("WARNING: You have a CUDA device, so you should probably run with --cuda")
    else:
        torch.cuda.manual_seed(args_seed)


In [ ]:
###############################################################################
# Load data
###############################################################################

corpus = data.Corpus(args_data)

eval_batch_size = 10
test_batch_size = 1

# should i use batchify here?
train_data = batchify2(corpus.train, args_batchSize, args_cuda)
val_data = batchify2(corpus.valid, eval_batch_size, args_cuda)
test_data = batchify2(corpus.test, test_batch_size, args_cuda)


In [ ]:
###############################################################################
# Build the model
###############################################################################

ntokens = len(corpus.dictionary)
model3 = model.RNNModel(args_model, ntokens, args_embedSize, args_nhid, args_nlayers, args_dropout, args_dropouth, args_dropouti, args_dropoute, args_wdrop, args_tied)
if args_cuda:
    model3.cuda()
total_params = sum(x.size()[0] * x.size()[1] if len(x.size()) > 1 else x.size()[0] for x in model3.parameters())
# print('Args:', args)
print('Model total parameters:', total_params)

criterion3 = nn.CrossEntropyLoss()


In [ ]:
###############################################################################
# Training code
###############################################################################

def disc_finetune(slant_lr):
    optimizer = torch.optim.SGD([
            {'params': model2.rnns[0].parameters(), 'lr': slant_lr/(2.6**2)},
            {'params': model2.rnns[1].parameters(), 'lr': slant_lr/2.6},
            {'params': model2.rnns[2].parameters(), 'lr': slant_lr}, 
            {'params': model2.lockdrop.parameters()},
            {'params': model2.idrop.parameters()},
            {'params': model2.hdrop.parameters()},
            {'params': model2.drop.parameters()},
            {'params': model2.encoder.parameters()},
            {'params': model2.decoder.parameters()}], lr=args_lr) 
    return optimizer

def slanTriangular(T, cut_frac, ratio, lrMax):
    # T = no: of epochs * no: of updates/epoch
    # cut_frac = 0.1
    # ratio=32
    # lrMax=0.01
    cut=math.floor(T*cut_frac)
    for t in range(1, T+1):
        if t<cut:
            p=t/cut
        else:
            p=1-((t-cut)/(cut*(1/(cut_frac-1))))
        lr_t=lrMax*((1+(p*(ratio-1)))/ratio)
        optim_finetune=disc_finetune(lr_t)
    
def evaluate(data_source, batch_size=10):
    # Turn on evaluation mode which disables dropout.
    if args_model == 'QRNN': model.reset()
    model.eval()
    total_loss = 0
    ntokens = len(corpus.dictionary)
    hidden = model.init_hidden(batch_size)
    for i in range(0, data_source.size(0) - 1, args_bptt):
        data, targets = get_batch2(data_source, i, args_bptt, evaluation=True)
        output, hidden = model3(data, hidden)
        output_flat = output.view(-1, ntokens)
        total_loss += len(data) * criterion(output_flat, targets).data
        hidden = repackage_hidden(hidden)
    print(total_loss.shape)
    return total_loss[0] / len(data_source)


def train():
    # Turn on training mode which enables dropout. 
    if args.model == 'QRNN': model.reset()
    total_loss = 0
    start_time = time.time()
    ntokens = len(corpus.dictionary)
    hidden = model.init_hidden(args.batch_size)
    batch, i = 0, 0
    while i < train_data.size(0) - 1 - 1:
        bptt = args.bptt if np.random.random() < 0.95 else args.bptt / 2.
        # Prevent excessively small or negative sequence lengths
        seq_len = max(5, int(np.random.normal(bptt, 5)))
        # There's a very small chance that it could select a very long sequence length resulting in OOM
        seq_len = min(seq_len, args.bptt + 10)

        lr2 = optimizer.param_groups[0]['lr']
        print("seq len")
        print(seq_len)
        optimizer.param_groups[0]['lr'] = lr2 * seq_len / args.bptt
        model.train()
        data, targets = get_batch(train_data, i, args, seq_len=seq_len)

        # Starting each batch, we detach the hidden state from how it was previously produced.
        # If we didn't, the model would try backpropagating all the way to start of the dataset.
        hidden = repackage_hidden(hidden)
        optimizer.zero_grad()

        output, hidden, rnn_hs, dropped_rnn_hs = model(data, hidden, return_h=True)
        raw_loss = criterion(output.view(-1, ntokens), targets)

        loss = raw_loss
        # Activiation Regularization
        loss = loss + sum(args.alpha * dropped_rnn_h.pow(2).mean() for dropped_rnn_h in dropped_rnn_hs[-1:])
        # Temporal Activation Regularization (slowness)
        loss = loss + sum(args.beta * (rnn_h[1:] - rnn_h[:-1]).pow(2).mean() for rnn_h in rnn_hs[-1:])
        loss.backward()

        # `clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
        torch.nn.utils.clip_grad_norm(model.parameters(), args.clip)
        optimizer.step()

        total_loss += raw_loss.data
        optimizer.param_groups[0]['lr'] = lr2
        if batch % args.log_interval == 0 and batch > 0:
            cur_loss = total_loss[0] / args.log_interval
            elapsed = time.time() - start_time
            print('| epoch {:3d} | {:5d}/{:5d} batches | lr {:02.2f} | ms/batch {:5.2f} | '
                    'loss {:5.2f} | ppl {:8.2f}'.format(
                epoch, batch, len(train_data) // args.bptt, optimizer.param_groups[0]['lr'],
                elapsed * 1000 / args.log_interval, cur_loss, math.exp(cur_loss)))
            total_loss = 0
            start_time = time.time()
        ###
        batch += 1
        i += seq_len


# Load the best saved model.
with open(args.save, 'rb') as f:
    model = torch.load(f)


In [ ]:
# Loop over epochs.
lr = args.lr
stored_loss = evaluate(val_data)
best_val_loss = []
# At any point you can hit Ctrl + C to break out of training early.
try:
    #optimizer = torch.optim.ASGD(model.parameters(), lr=args.lr, weight_decay=args.wdecay)
    optimizer = torch.optim.ASGD(model.parameters(), lr=args.lr, t0=0, lambd=0., weight_decay=args.wdecay)
    for epoch in range(1, args.epochs+1):
        epoch_start_time = time.time()
        train()
        if 't0' in optimizer.param_groups[0]:
            tmp = {}
            for prm in model.parameters():
                tmp[prm] = prm.data.clone()
                prm.data = optimizer.state[prm]['ax'].clone()

            val_loss2 = evaluate(val_data)
            print('-' * 89)
            print('| end of epoch {:3d} | time: {:5.2f}s | valid loss {:5.2f} | '
                    'valid ppl {:8.2f}'.format(epoch, (time.time() - epoch_start_time),
                                               val_loss2, math.exp(val_loss2)))
            print('-' * 89)

            if val_loss2 < stored_loss:
                with open(args.save, 'wb') as f:
                    torch.save(model, f)
                print('Saving Averaged!')
                stored_loss = val_loss2

            for prm in model.parameters():
                prm.data = tmp[prm].clone()

        if (len(best_val_loss)>args.nonmono and val_loss2 > min(best_val_loss[:-args.nonmono])):
            print('Done!')
            import sys
            sys.exit(1)
            optimizer = torch.optim.ASGD(model.parameters(), lr=args.lr, t0=0, lambd=0., weight_decay=args.wdecay)
            #optimizer.param_groups[0]['lr'] /= 2.
        best_val_loss.append(val_loss2)

except KeyboardInterrupt:
    print('-' * 89)
    print('Exiting from training early')

# Load the best saved model.
with open(args.save, 'rb') as f:
    model = torch.load(f)
    
# Run on test data.
test_loss = evaluate(test_data, test_batch_size)
print('=' * 89)
print('| End of training | test loss {:5.2f} | test ppl {:8.2f}'.format(
    test_loss, math.exp(test_loss)))
print('=' * 89)